# LightGBM model

### Import libraries

In [1]:
import sys

# go up two directories
sys.path.append('../../../')

# Import necessary functions from codecompasslib
from codecompasslib.models.lightgbm_model import generate_lightGBM_recommendations, load_data

c:\Users\ketis\anaconda3\envs\codecompassvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token loaded successfully.


### Load the datasets

In [2]:
full_data_folder_id = '1Qiy9u03hUthqaoBDr4VQqhKwtLJ2O3Yd'
full_data_embedded_folder_id = '139wi78iRzhwGZwxmI5WALoYocR-Rk9By'

df_non_embedded, df_embedded = load_data(full_data_folder_id, full_data_embedded_folder_id)


Download 11%.

Download 23%.

Download 35%.

Download 47%.

Download 59%.

Download 71%.

Download 83%.

Download 95%.

Download 100%.


c:\Users\ketis\UniversityStuff\2024\RecAndChat\CodeCompass\codecompasslib\models\examples\../../..\codecompasslib\API\drive_operations.py:88: DtypeWarning: Columns (6,11,12,15,16,17,18,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(fh)



Download 4%.

Download 8%.

Download 12%.

Download 16%.

Download 20%.

Download 24%.

Download 28%.

Download 32%.

Download 36%.

Download 40%.

Download 44%.

Download 49%.

Download 53%.

Download 57%.

Download 61%.

Download 65%.

Download 69%.

Download 73%.

Download 77%.

Download 81%.

Download 85%.

Download 89%.

Download 94%.

Download 98%.

Download 100%.
Data loaded


### Utilize LightGBM model

In [3]:
# choosing a target user to generate recommendations for
target_user = 'Rameshwar0852'

In [9]:
# Generate recommendations for the target user
recommendations = generate_lightGBM_recommendations(target_user, df_non_embedded, df_embedded, number_of_recommendations=10)

Training LightGBM model
[LightGBM] [Info] Number of positive: 8, number of negative: 976063
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.516589 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 65768
[LightGBM] [Info] Number of data points in the train set: 976071, number of used features: 258
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000008 -> initscore=-11.711841
[LightGBM] [Info] Start training from score -11.711841
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[7]	valid_0's auc: 0.873331


In [11]:
# Display the recommendations
for index, repo in enumerate(recommendations):
    name = df_non_embedded[df_non_embedded['id'] == repo[0]]['name'].values[0]
    description = df_non_embedded[df_non_embedded['id'] == repo[0]]['description'].values[0]
    link = f"https://github.com/{repo[1]}/{name}"
    
    print(f"Recommendation {index + 1}:")
    print(f"Name: {name}")
    print(f"Description: {description}")
    print(f"Link: {link}")

Recommendation 1:
Name: Back-Front_WebCodeTricks
Description: Projeto de TG
Link: https://github.com/adolfosp/Back-Front_WebCodeTricks
Recommendation 2:
Name: Peerage
Description: A quick coat-of-arms (or blazon) generator
Link: https://github.com/r2d2m/Peerage
Recommendation 3:
Name: Loteria
Description: Apuração de jogos de loteria.
Link: https://github.com/MelqSantos/Loteria
Recommendation 4:
Name: Web_Learning
Description: Aprendiendo a programar con el grone
Link: https://github.com/mefsaal/Web_Learning
Recommendation 5:
Name: sack
Description: s(hortcut)-ack: a faster way to use ag, ack (or grep)!
Link: https://github.com/hwgao/sack
Recommendation 6:
Name: computer-vision
Description: Hacker's guide to computer vision
Link: https://github.com/batermj/computer-vision
Recommendation 7:
Name: BabyNamer
Description: A tool to help you choose a baby name
Link: https://github.com/cortesoft/BabyNamer
Recommendation 8:
Name: repaso
Description: programa para generar 5 numeros aleatorios
